# Tutorial

<b>Warning: it is necessary to use <u>setups='sg15'</u> in GPAW</b> (ONCV pseudopotentials)

In [ ]:
from gpaw import GPAW, FermiDirac
from gpaw.wavefunctions.pw import PW
from ase.io import read
from TDDFT import TDDFT
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

c = 20
PW_cut=400
nbands=40
atoms = read('hBN.cif')
atoms.cell[2,2]=c
atoms.center()

calc = GPAW(mode=PW(PW_cut),xc='PBE',
            kpts={'size': (8, 8, 1), 'gamma': True},
            setups='sg15',nbands=nbands*2,
            occupations=FermiDirac(0.0001),
            convergence={'bands':nbands},
            txt='calc.txt')

atoms.set_calculator(calc)
atoms.get_potential_energy()
calc.write('gs.gpw')

/home/yzhumagulov/anaconda3/lib/python3.7/site-packages/mkl_fft/_numpy_fft.py:244: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  output = mkl_fft.ifft(a, n, axis)


# Initialization 

In [ ]:
tddft=TDDFT(calc,nbands)

In [ ]:
import pickle
with open('save', 'wb') as f:
    pickle.dump(tddft,f)

# Calculation of the dipole matrix
Сalculation of the dipole matrix occurs according to the following equation
$$ d_{nm}(k)=\int_\Omega dr \; u_{kn}^{*}(r) \; r \; u_{km}(r)$$
where $u_{kn}(r)$ - periodic part of Kohn-Sham wavefunction which is stored in <b>TDDFT.ukn</b>

$r$ - coordinate inside the elementary cell which is stored in <b>TDDFT.r</b>

$\Omega$ - volume of the elementary cell which is stored in <b>TDDFT.volume</b>

In [ ]:
direction=[0,0,1]
dipole=tddft.get_dipole_matrix(direction)
plt.title("Dipole matrix at K=0")
plt.imshow(np.abs(dipole[0]))
plt.colorbar()

# Calculation of the Hartree matrix
Calculation of the Hartree matrix occurs according to following equation
$$ V^{H}_{nm}(k)=\int_\Omega dr \; u_{kn}^{*}(r) \; V_H(r) \; u_{km}(r)$$
$V_H(r)$ - Hartree potential which is obtained by solving Poisson equation using folowing equations:
$$n(G)=FFT(n(r)) \Rightarrow V_H(G)=-4\pi\frac{n(G)}{|G|^2} \Rightarrow V_H(r)=IFFT(V_H(G))$$
$$n(r)=2 \sum_{k}^{IBZ} w(k)\sum_n^{N_b}f_n\sum_m^{N_b} \left|c_{nm}(k)u_{km}(r)\right|^2$$
where $IBZ$ -irreducible Brillioun zone

$w(k)$ - weight of k-points in irreducible Brillioun zone which is stored in <b>TDDFT.w</b>

$f_n$ - occupation of Kohn-Sham orbitals which is stored in <b>TDDFT.f</b>

$c_{nm}(k)$ - wavefunction in second-quantization basis

In [ ]:
VH_matrix=tddft.get_Hartree_matrix()
plt.title("Hartree matrix at K=0")
plt.imshow(VH_matrix[0].real)
plt.colorbar()

# Calculation of the Fock matrix
Calculation of the Fock matrix occurs according to following equation
$$ V^{F}_{nm}(k)=-\sum^{occ}_l \sum_{q}^{BZ} \sum_{G} M^{*}_{ln}(k,q,G)M_{l,m}(k,q,G)v(q+G)$$
where $M_{n,m}(k,q,G)$ - pair-density which is stored in <b>TDDFT.M</b>
$$M_{n,m}(k,q,G)=IFFT\left(u_{k+q,n}^{*}(r)u_{k,n}(r)\right)$$
and $v(q+G)$ - Coloumb potential
$$v(q+G)=\frac{4\pi}{\left|q+G\right|^{2}}$$

In [ ]:
VF_matrix=tddft.get_Fock_matrix()
plt.imshow(VF_matrix[0].real)
plt.colorbar()

# Calculation of the LDA exchange matrix
Calculation of the LDA exchange matrix matrix occurs according to following equation
$$ V^{LDAx}_{nm}(k)=\int_\Omega dr \; u_{kn}^{*}(r) \; \left(-\frac{3n(r)}{\pi}\right)^{1/3} \; u_{km}(r)$$

In [ ]:
LDAx_matrix=tddft.get_LDA_exchange_matrix()
plt.imshow(LDAx_matrix[0].real)
plt.colorbar()

# Calculation of the LDA correlation matrix
Calculation of the LDA correlation matrix matrix occurs according to following equation
$$ V^{LDAx}_{nm}(k)=\int_\Omega dr \; u_{kn}^{*}(r) \; f_c(n(r)) \; u_{km}(r)$$
$fc(n(r))$ parametrization taken from THE JOURNAL OF CHEMICAL PHYSICS 145, 157101 (2016)

In [ ]:
LDAc_matrix=tddft.get_LDA_correlation_matrix()
plt.imshow(LDAc_matrix[0].real)
plt.colorbar()

# TDDFT

In [ ]:
steps=1000;dt=0.1
E=np.zeros(steps);time=np.arange(steps)*dt
E[0]=0.1
result=tddft.propagate(dt=dt,steps=steps,E=E,operator=dipole)

In [ ]:
for n in range(tddft.nbands):
    plt.plot(time,result[:,n].real,label=str(n))
plt.grid()
plt.legend(loc='best')

In [ ]:
from ase.units import Hartree, Bohr
spectrum=np.fft.fft(tddft.macro_dipole)
freq = np.fft.fftfreq(steps, d=dt)*Hartree
spectrum=spectrum[np.argsort(freq)]
freq=np.sort(freq)

In [ ]:
from itertools import product
#объем элементарной ячейки
volume = np.abs(np.linalg.det(calc.wfs.gd.cell_cv)) 
#редуцированная зона Бриллюэна
K=calc.get_ibz_k_points();NK=K.shape[0] 
#веса точек редуцированной зоны Бриллюэна
wk=calc.get_k_point_weights()
#энергия орбиталей Кона-Шэма
EK=[calc.get_eigenvalues(k) for k in range(NK)] 
EK=np.array(EK)/Hartree
#Число валентных электронов
nvalence=int(calc.occupations.nvalence/2)
nbands=calc.get_number_of_bands()
#Валентные зоны
vb=np.arange(0,nvalence/2,dtype=int)
#Зоны проводимости
cb=np.arange(nvalence/2,nbands,dtype=int)
#Индексы комбинаций оптических электрон-дырочных пар
indexes=[(k,v,c) for k,v,c in product(range(NK),range(len(vb)),range(len(cb)))]
NH=len(indexes)

norm=Bohr**3*calc.wfs.gd.dv

In [ ]:
from tqdm import tqdm
velocity=np.zeros((NH),dtype=np.complex)
omega=np.linspace(0,10,20000)/Hartree;eta=0.05/Hartree
chi=np.zeros(omega.size,dtype=np.complex)
for indx in range(NH):
    k,v,c=indexes[indx]
    wf1=calc.wfs.get_wave_function_array(n=vb[v],k=k,s=0,realspace=False)
    wf2=calc.wfs.get_wave_function_array(n=cb[c],k=k,s=0,realspace=False)
    G_=calc.wfs.pd.get_reciprocal_vectors(q=k,add_q=True)
    velocity[indx]=calc.wfs.pd.integrate(wf1,G_[:,2]*wf2)/(EK[k,cb[c]]-EK[k,vb[v]])
    chi+=2*wk[k]*np.abs(velocity[indx])**2/(omega+1j*eta-EK[k,cb[c]]+EK[k,vb[v]])
    chi-=2*wk[k]*np.abs(velocity[indx])**2/(omega+1j*eta+EK[k,cb[c]]-EK[k,vb[v]])
chi/=(volume)
epsilon=1-4*np.pi*chi

In [ ]:
plt.plot(omega*Hartree,epsilon.imag)
plt.plot(freq,np.abs(spectrum),'o')
plt.xlim([0,10])
plt.ylim([0,0.2])
plt.grid()